In [11]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from abc import ABC, abstractmethod

### Clase abstracta para el procesamiento de la imagen

In [12]:
class Modulacion(ABC):
    """
    Clase abstracta que define los métodos para la modulación y demodulación de señales.
    """

    @abstractmethod
    def modulate(self, bits, fc, mpp):
        """
        Método abstracto para la modulación de una señal.

        Parámetros:
        - bits: una secuencia de bits a ser modulados.
        - fc: frecuencia de la portadora.
        - mpp: muestras por periodo de la portadora.
        """
        pass

    @abstractmethod
    def demodulate(self, **kwargs):
        """
        Método abstracto para la demodulación de una señal.

        Parámetros:
        - kwargs: argumentos adicionales necesarios para la demodulación.
        """
        pass


### Clase para la modulación BPSK

In [13]:
class BPSKStrategy(Modulacion):
    """
    Clase que implementa la estrategia de modulación BPSK (Binary Phase Shift Keying).

    Métodos:
        - modulate(bits, fc, mpp): Realiza la modulación BPSK de una secuencia de bits.
        - demodulate(senal_Rx, portadora, mpp, fc): Realiza la demodulación BPSK de una señal recibida.

    Atributos heredados:
        - ...

    """

    def modulate(self, bits, fc, mpp):
        """
        Realiza la modulación BPSK de una secuencia de bits.

        Parámetros:
            - bits (array): Secuencia de bits a modular.
            - fc (float): Frecuencia de la portadora.
            - mpp (int): Muestras por periodo de la portadora.

        Retorna:
            - senal_Tx (array): Señal modulada BPSK.
            - Pm (float): Potencia media de la señal modulada.
            - portadora (array): Señal de la portadora.

        """

        N = len(bits)
        Tc = 1 / fc
        t_periodo = np.linspace(0, Tc, mpp)
        portadora = np.sin(2 * np.pi * fc * t_periodo)
        t_simulacion = np.linspace(0, N * Tc, N * mpp)
        senal_Tx = np.zeros(t_simulacion.shape)
        for i, bit in enumerate(bits):
            senal_Tx[i * mpp:(i + 1) * mpp] = portadora if bit == 1 else -portadora
        Pm = (1 / (N * Tc)) * np.trapezoid(np.power(senal_Tx, 2), t_simulacion)
        return senal_Tx, Pm, portadora

    def demodulate(self, senal_Rx, portadora, mpp):
        """
        Demodula una señal recibida mediante el esquema BPSK.

        Parámetros:
        - senal_Rx (array): La señal recibida a demodular.
        - portadora (array): La señal de la portadora utilizada en la modulación.
        - mpp (int): Número de muestras por periodo de la portadora.
        - fc (float): Frecuencia de la portadora.

        Retorna:
        - bits_Rx (array): Los bits demodulados de la señal recibida.
        """
        M = len(senal_Rx)
        N = int(M / mpp)
        bits_Rx = np.zeros(N)
        Es = np.sum(portadora ** 2)
        for i in range(N):
            producto = senal_Rx[i * mpp:(i + 1) * mpp] * portadora
            Ep = np.sum(producto)
            bits_Rx[i] = 1 if Ep > Es * 0.8 else 0
        return bits_Rx.astype(int)

### Clase para la modulacion QPSK

In [14]:
class QPSKStrategy(Modulacion):
    """
    Clase que implementa la estrategia de modulación y demodulación QPSK.
    """

    def modulate(self, bits, fc, mpp): 
        """
        Realiza la modulación QPSK de una secuencia de bits.

        Parámetros:
        - bits: Secuencia de bits a modular.
        - fc: Frecuencia de la portadora.
        - mpp: Muestras por periodo de la portadora.

        Retorna:
        - senal_Tx: Señal modulada QPSK.
        - Pm: Potencia media de la señal modulada.
        - (portadora_I, portadora_Q): Tupla con las portadoras I y Q.
        """
        N = len(bits) // 2
        Tc = 1 / fc
        t_periodo = np.linspace(0, Tc, mpp)
        portadora_I = np.sin(2 * np.pi * fc * t_periodo)
        portadora_Q = np.cos(2 * np.pi * fc * t_periodo)
        t_simulacion = np.linspace(0, N * Tc, N * mpp)
        senal_Tx = np.zeros(t_simulacion.shape)
        
        for i in range(N):
            bit_I = bits[2*i]
            bit_Q = bits[2*i + 1]
            senal_Tx[i * mpp:(i + 1) * mpp] = (portadora_I if bit_I == 1 else -portadora_I) + \
                                            (portadora_Q if bit_Q == 1 else -portadora_Q)
        
        Pm = (1 / (N * Tc)) * np.trapezoid(np.power(senal_Tx, 2), t_simulacion)
        return senal_Tx, Pm, (portadora_I, portadora_Q) 

    def demodulate(self, senal_Rx, portadora_I, portadora_Q, mpp):
        """
        Demodula una señal recibida utilizando la técnica de detección de energía.
        
        Parámetros:
        - senal_Rx (array): La señal recibida a demodular.
        - portadora_I (array): La portadora en fase I utilizada en la modulación.
        - portadora_Q (array): La portadora en fase Q utilizada en la modulación.
        - mpp (int): Número de muestras por periodo de la portadora.
        
        Retorna:
        - bits_Rx (array): Los bits demodulados de la señal recibida.
        """
        
        M = len(senal_Rx)
        N = int(M / mpp)
        bits_Rx = np.zeros(2 * N)
        Es = np.sum(portadora_I ** 2)
        
        for i in range(N):
            segmento = senal_Rx[i * mpp:(i + 1) * mpp]
            producto_I = segmento * portadora_I
            producto_Q = segmento * portadora_Q
            Ep_I = np.sum(producto_I)
            Ep_Q = np.sum(producto_Q)
            
            bits_Rx[2*i] = 1 if Ep_I > Es * 0.8 else 0
            bits_Rx[2*i + 1] = 1 if Ep_Q > Es * 0.8 else 0
        
        return bits_Rx.astype(int)
        
        

### Clase del Factory

In [15]:
class ModulacionFactory:
    """
    Clase que representa una fábrica de estrategias de modulación.
    """

    strategy_map_modulation = {
        "bpsk": BPSKStrategy(),
        "qpsk": QPSKStrategy(),
    }

    @classmethod
    def get_strategy(cls, strategy_name: str):
        """
        Obtiene la estrategia de modulación correspondiente al nombre de estrategia dado.

        Args:
            strategy_name (str): El nombre de la estrategia de modulación.

        Returns:
            Modulacion: La estrategia de modulación correspondiente.

        Raises:
            ValueError: Si no se encuentra la estrategia de modulación.
        """
        return cls.strategy_map_modulation.get(strategy_name.lower(), None)
    
    @classmethod
    def modulation(cls, strategy_name, bits, fc, mpp):
        """
        Realiza la modulación utilizando la estrategia de modulación correspondiente al nombre de estrategia dado.

        Args:
            strategy_name (str): El nombre de la estrategia de modulación.
            bits (list): La lista de bits a modular.
            fc (float): La frecuencia de la portadora.
            mpp (float): Los muestras por periodo.

        Returns:
            list: La señal modulada.

        Raises:
            ValueError: Si no se encuentra la estrategia de modulación.
        """
        strategy: Modulacion = cls.get_strategy(strategy_name)
        if strategy is None:
            raise ValueError(f"Strategy {strategy_name} not found")
        return strategy.modulate(bits, fc, mpp)
    
    @classmethod
    def demodulation(cls, strategy_name, **kwargs):
        """
        Realiza la demodulación utilizando la estrategia de modulación correspondiente al nombre de estrategia dado.

        Args:
            strategy_name (str): El nombre de la estrategia de modulación.
            **kwargs: Argumentos adicionales para la demodulación.

        Returns:
            object: El resultado de la demodulación.

        Raises:
            ValueError: Si no se encuentra la estrategia de modulación.
        """
        strategy: Modulacion = cls.get_strategy(strategy_name)
        if strategy is None:
            raise ValueError(f"Strategy {strategy_name} not found")
        return strategy.demodulate(**kwargs)

### Funciones adicionales

In [16]:
def open_image(img_path: str) -> np.ndarray:
    """
    Abre una imagen y la convierte en un arreglo de numpy.

    Parámetros:
    img_path (str): La ruta de la imagen a abrir.

    Retorna:
    numpy.ndarray: El arreglo de numpy que representa la imagen.
    """
    img = Image.open(img_path)
    return np.array(img)

def rgb_a_bit(imagen: np.ndarray) -> np.ndarray:
    """
    Convierte una imagen RGB en una secuencia de bits.

    Parámetros:
    imagen (numpy.ndarray): La imagen RGB a convertir.

    Retorna:
    numpy.ndarray: La secuencia de bits resultante.
    """
    x, y, z = imagen.shape
    pixeles = np.reshape(imagen, (x * y * z,))
    bits = [format(pixel, '08b') for pixel in pixeles]
    bits_Rx = np.array(list(''.join(bits)))
    return bits_Rx.astype(int)

def canal_ruidoso(senal_Tx: np.ndarray, Pm: float, SNR: float) -> np.ndarray:
    """
    Simula un canal ruidoso al agregar ruido a una señal transmitida.

    Parámetros:
    - senal_Tx (array): La señal transmitida.
    - Pm (float): La potencia de la señal transmitida.
    - SNR (float): La relación señal-ruido en decibelios.

    Retorna:
    - senal_Rx (array): La señal recibida con ruido.
    """
    
    Pn = Pm / (10 ** (SNR / 10))
    ruido = np.random.normal(0, np.sqrt(Pn), senal_Tx.shape)
    senal_Rx = senal_Tx + ruido
    return senal_Rx

def bits_a_rgb(bits_Rx: np.ndarray, dimensiones: tuple) -> np.ndarray:
    """
    Convierte una secuencia de bits en una matriz de píxeles RGB.

    Args:
        bits_Rx (np.ndarray): Secuencia de bits a convertir.
        dimensiones (tuple): Dimensiones de la matriz de píxeles resultante.

    Returns:
        np.ndarray: Matriz de píxeles RGB resultante.

    Raises:
        ValueError: Si la longitud de los bits no coincide con el tamaño esperado.
    """
    bits = np.array(bits_Rx)
    N = int(np.prod(dimensiones))
    if len(bits) != N * 8:
        raise ValueError("La longitud de los bits no coincide con el tamaño esperado.")
    bits = np.split(bits, N)
    canales = [int(''.join(map(str, canal)), 2) for canal in bits]
    pixeles = np.reshape(canales, dimensiones)
    return pixeles.astype(np.uint8)

def ssim(img1, img2):
    """
    Calcula el Índice de Similitud Estructural (SSIM) entre dos imágenes.
    
    Parámetros:
    img1 (numpy.ndarray): Primera imagen a comparar.
    img2 (numpy.ndarray): Segunda imagen a comparar.
    
    Retorna:
    float: Valor del SSIM entre las dos imágenes.
    """
    
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2
    
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = np.ones((8,8))
    
    window = np.outer(kernel, kernel)
    
    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # media
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2
    
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    
    return ssim_map.mean()

In [21]:
fc = 5000
mpp = 20
SNR = 5


imagen_Tx = open_image("images/image.png")
dimensiones = imagen_Tx.shape

inicio = time.time()
bits_Tx = rgb_a_bit(imagen_Tx)
senal_Tx_BPSK, Pm_BPSK, portadora_BPSK = ModulacionFactory().modulation("bpsk", bits_Tx, fc, mpp)
senal_Tx_QPSK, Pm_QPSK, (portadora_I_QPSK, portadora_Q_QPSK) = ModulacionFactory().modulation("qpsk", bits_Tx, fc, mpp)

#añadir ruido
senal_Rx_QPSK = canal_ruidoso(senal_Tx_QPSK, Pm_QPSK, SNR)
senal_Tx_BPSK = canal_ruidoso(senal_Tx_BPSK, Pm_BPSK, SNR)

#demodular
bits_Rx_BPSK = ModulacionFactory().demodulation("bpsk", senal_Rx=senal_Tx_BPSK, portadora=portadora_BPSK, mpp=mpp)
bits_Rx_QPSK = ModulacionFactory().demodulation("qpsk", senal_Rx=senal_Rx_QPSK, portadora_I=portadora_I_QPSK, portadora_Q=portadora_Q_QPSK, mpp=mpp)

# Reconstruir imagen
imagen_Rx_QPSK = bits_a_rgb(bits_Rx_QPSK, dimensiones)
imagen_Rx_BPSK = bits_a_rgb(bits_Rx_BPSK, dimensiones)

fin = time.time()

tiempo_ejecucion = fin - inicio

if imagen_Tx.shape != imagen_Rx_QPSK.shape:
    imagen_Rx_QPSK = cv2.resize(imagen_Rx_QPSK, (imagen_Tx.shape[1], imagen_Tx.shape[0]))

if imagen_Tx.shape != imagen_Rx_BPSK.shape:
    imagen_Rx_BPSK = cv2.resize(imagen_Rx_BPSK, (imagen_Tx.shape[1], imagen_Tx.shape[0]))

plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(imagen_Tx)
plt.title('Transmitido')

plt.subplot(2, 2, 2)
plt.imshow(imagen_Rx_BPSK)
plt.title('Recuperado (BPSK)')

plt.subplot(2, 2, 3)
plt.imshow(imagen_Rx_QPSK)
plt.title('Recuperado (QPSK)')

plt.show()

errores_BPSK = sum(abs(bits_Tx - bits_Rx_BPSK))
BER_BPSK = errores_BPSK / len(bits_Tx)
print('{} errores (BPSK), para un BER de {:0.4f}.'.format(errores_BPSK, BER_BPSK))

errores_QPSK = sum(abs(bits_Tx - bits_Rx_QPSK))
BER_QPSK = errores_QPSK / len(bits_Tx)
print('{} errores (QPSK), para un BER de {:0.4f}.'.format(errores_QPSK, BER_QPSK))

# Visualización del BER
plt.figure(figsize=(6, 4))
plt.bar(['BPSK', 'QPSK'], [BER_BPSK, BER_QPSK])
plt.ylabel('Bit Error Rate')
plt.title('Comparación del Bit Error Rate entre BPSK y QPSK')
plt.show()

# Cálculo de la similitud de imagen
ssim_bpsk = ssim(imagen_Tx, imagen_Rx_BPSK)
ssim_qpsk = ssim(imagen_Tx, imagen_Rx_QPSK)

print(f'SSIM (BPSK): {ssim_bpsk}')
print(f'SSIM (QPSK): {ssim_qpsk}')

# Visualización de la similitud de imagen
plt.figure(figsize=(6, 4))
plt.bar(['BPSK', 'QPSK'], [ssim_bpsk, ssim_qpsk])
plt.ylabel('SSIM')
plt.title('Comparación de Similitud de Imagen entre BPSK y QPSK')
plt.show()






In [ ]:
# Parámetros
fc = 5000  # frecuencia de la portadora
mpp = 20   # muestras por periodo de la portadora
SNR = 5    # relación señal-a-ruido del canal

inicio = time.time()

# 1. Importar y convertir la imagen a transmitir
imagen_Tx = fuente_info(r"C:\Users\jrinc\OneDrive\Escritorio\Redes\Imag2.jpg")
dimensiones = imagen_Tx.shape

# 2. Codificar los píxeles de la imagen
bits_Tx = rgb_a_bit(imagen_Tx)

# 3. Modular la cadena de bits usando el esquema BPSK
senal_Tx_BPSK, Pm_BPSK, portadora_BPSK = modulador_bpsk(bits_Tx, fc, mpp)

# 4. Modular la cadena de bits usando el esquema QPSK
senal_Tx_QPSK, Pm_QPSK, (portadora_I_QPSK, portadora_Q_QPSK) = modulador_qpsk(bits_Tx, fc, mpp)

# 5. Transmitir la señal modulada por un canal ruidoso
senal_Rx_BPSK = canal_ruidoso(senal_Tx_BPSK, Pm_BPSK, SNR)
senal_Rx_QPSK = canal_ruidoso(senal_Tx_QPSK, Pm_QPSK, SNR)

# 6. Demodular la señal recibida
bits_Rx_BPSK = demodulador_bpsk(senal_Rx_BPSK, portadora_BPSK, mpp)
bits_Rx_QPSK = demodulador_qpsk(senal_Rx_QPSK, portadora_I_QPSK, portadora_Q_QPSK, mpp)

# 7. Reconstruir la imagen
imagen_Rx_BPSK = bits_a_rgb(bits_Rx_BPSK, dimensiones)
imagen_Rx_QPSK = bits_a_rgb(bits_Rx_QPSK, dimensiones)

# Asegurarse de que las imágenes tengan las mismas dimensiones
if imagen_Tx.shape != imagen_Rx_BPSK.shape:
    imagen_Rx_BPSK = cv2.resize(imagen_Rx_BPSK, (imagen_Tx.shape[1], imagen_Tx.shape[0]))

if imagen_Tx.shape != imagen_Rx_QPSK.shape:
    imagen_Rx_QPSK = cv2.resize(imagen_Rx_QPSK, (imagen_Tx.shape[1], imagen_Tx.shape[0]))

# Mostrar la imagen transmitida y recibida
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(imagen_Tx)
plt.title('Transmitido')

plt.subplot(2, 2, 2)
plt.imshow(imagen_Rx_BPSK)
plt.title('Recuperado (BPSK)')

plt.subplot(2, 2, 3)
plt.imshow(imagen_Rx_QPSK)
plt.title('Recuperado (QPSK)')

plt.show()

# Cálculo del tiempo de simulación y tasa de error
print('Duración de la simulación:', time.time() - inicio)

errores_BPSK = sum(abs(bits_Tx - bits_Rx_BPSK))
BER_BPSK = errores_BPSK / len(bits_Tx)
print('{} errores (BPSK), para un BER de {:0.4f}.'.format(errores_BPSK, BER_BPSK))

errores_QPSK = sum(abs(bits_Tx - bits_Rx_QPSK))
BER_QPSK = errores_QPSK / len(bits_Tx)
print('{} errores (QPSK), para un BER de {:0.4f}.'.format(errores_QPSK, BER_QPSK))

# Visualización del BER
plt.figure(figsize=(6, 4))
plt.bar(['BPSK', 'QPSK'], [BER_BPSK, BER_QPSK])
plt.ylabel('Bit Error Rate')
plt.title('Comparación del Bit Error Rate entre BPSK y QPSK')
plt.show()

# Cálculo de la similitud de imagen
ssim_bpsk = ssim(imagen_Tx, imagen_Rx_BPSK)
ssim_qpsk = ssim(imagen_Tx, imagen_Rx_QPSK)

print(f'SSIM (BPSK): {ssim_bpsk}')
print(f'SSIM (QPSK): {ssim_qpsk}')

# Visualización de la similitud de imagen
plt.figure(figsize=(6, 4))
plt.bar(['BPSK', 'QPSK'], [ssim_bpsk, ssim_qpsk])
plt.ylabel('SSIM')
plt.title('Comparación de Similitud de Imagen entre BPSK y QPSK')
plt.show()


ModuleNotFoundError: No module named 'cv2'